In [25]:
import ee
import geemap

In [26]:
ee.Initialize()

In [27]:
from datetime import datetime

In [28]:
Map = geemap.Map()

In [29]:
def milsec2date(millsec_list: list, no_duplicate = False)->list:
  '''
  Input
  ---
  this function takes `imgcollection.aggregate_array('system:time_start')` which is a list of milliseconds dates as input

  Reutrns
  ---
  * Defult: a list of dates in GEE date string format
  * No_duplicate: returns the list of dates but removes the duplicates
    '''
  if no_duplicate:
    date = [datetime.fromtimestamp(t/1000.0).strftime('%Y-%m-%d') for t in millsec_list]
    date_no_duplicate = list(dict.fromkeys(date))
    return  date_no_duplicate
  else:
    date = [datetime.fromtimestamp(t/1000.0).strftime('%Y-%m-%d') for t in millsec_list] 
    return date

In [30]:
def sen2_print(s2_collection):
  '''
  prints the important properites of sentienel 2 data collection
  '''
  print('CLOUD_SHADOW_PERCENTAGE  :',s2_collection.aggregate_array('CLOUD_SHADOW_PERCENTAGE').getInfo())
  print('VEGETATION_PERCENTAGE    :',s2_collection.aggregate_array('VEGETATION_PERCENTAGE').getInfo())
  print('NOT_VEGETATED_PERCENTAGE :',s2_collection.aggregate_array('NOT_VEGETATED_PERCENTAGE').getInfo())

  # print('SENSOR_QUCLOUD_COVERAGE_ASSESSMENTALITY :',s2_collection.aggregate_array('CLOUD_COVERAGE_ASSESSMENT').getInfo())
  # print('GENERATION_TIME          :',s2_collection.aggregate_array('GENERATION_TIME').getInfo())
  # print('SENSING_ORBIT_NUMBER     :',s2_collection.aggregate_array('SENSING_ORBIT_NUMBER').getInfo())
  print('NODATA_PIXEL_PERCENTAGE  :',s2_collection.aggregate_array('NODATA_PIXEL_PERCENTAGE').getInfo())
  # print('DATATAKE_TYPE            :',s2_collection.aggregate_array('DATATAKE_TYPE').getInfo())
  # print('SENSING_ORBIT_NUMBER     :',s2_collection.aggregate_array('SENSING_ORBIT_NUMBER').getInfo())
  print('SNOW_ICE_PERCENTAGE      :',s2_collection.aggregate_array('SNOW_ICE_PERCENTAGE').getInfo())
  # print('THIN_CIRRUS_PERCENTAGE   :',s2_collection.aggregate_array('THIN_CIRRUS_PERCENTAGE').getInfo())
  print('WATER_PERCENTAGE         :',s2_collection.aggregate_array('WATER_PERCENTAGE').getInfo())
  print('Date                     :',milsec2date(s2_collection.aggregate_array('system:time_start').getInfo()))
  # print('system:band_names        :',s2_collection.aggregate_array('system:band_names').getInfo())

In [31]:
s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

In [ ]:
import rois

In [ ]:
date_range = ('2017-05-20','2017-08-10')
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate(date_range[0],date_range[1]) \
    .filterBounds(rois.ROI)
s2.size()

In [62]:
date_range = ('2017-05-20','2017-08-10')
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                      .filterDate(date_range[0], date_range[1]) \
                      .filterBounds(rois)
s2.size()

EEException: Invalid GeoJSON geometry.

In [58]:
date_range = ('2017-05-20','2017-08-10')
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                      .filterDate(date_range[0], date_range[1]) \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                      .filter(ee.Filter.lt('SNOW_ICE_PERCENTAGE',10)) \
                      .filterBounds(rois.rio)
s2.size()

In [41]:
# Initialize Earth Engine
ee.Initialize()

# Define the ROI (Region of Interest)
roi = ee.Geometry.Point([-122.084, 37.42]).buffer(1000)

# Define the date range
start_date = '2021-01-01'
end_date = '2021-12-31'

# Load Sentinel-2 data for the ROI and date range
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(rois.rio) \
    .filterDate(start_date, end_date)

# Select the bands to visualize
bands = ['B4', 'B3', 'B2']

# Create a median composite image
image = collection.median().select(bands)

# Set visualization parameters for the image
viz_params = {'bands': bands, 'min': 0, 'max': 3000}

# Create a Map instance
Map = geemap.Map()

# Add the image to the Map
Map.addLayer(image, viz_params, 'Sentinel-2')

# Add the ROI to the Map
Map.addLayer(roi, {'color': 'red'}, 'ROI')

# Display the Map
Map.centerObject(roi, 12)
Map.addLayerControl()
Map

Map(center=[37.420001025890286, -122.08399996417683], controls=(WidgetControl(options=['position', 'transparen…

In [42]:
sen2_print(collection)

CLOUD_SHADOW_PERCENTAGE  : [0.088712, 0.247935, 0.373204, 0.028447, 0.02504, 0.022233, 0.025916, 0.052273, 0, 0.000786, 1.601096, 0.060647, 0.031924, 0.041548, 0.906603, 0.943759, 0.045962, 0.138271, 0.238762, 1.268828, 0.037276, 0.156559, 0.047299, 2.526806, 0.329783, 0.05719, 0.272474, 1.996696, 0.003623, 0.213417, 0.641033, 1.093062, 2.035723, 0.642174, 0.001516, 0.07575, 1.716371, 0.118925, 0.040076, 1.191675, 0.054064, 1.452882, 2.468953, 1.460606, 0.19904, 0.113507, 0.023198, 1.239754, 0.222939, 0.004127, 0.054917, 1.440536, 0.07286, 0.276349, 0, 0, 0.414015, 0.007225, 0.000179, 0.003006, 0.000116, 0.017021, 0, 0.621329, 0, 0.148132, 0.021088, 0.031622, 0.381041, 0.099207, 0.287381, 1.083288, 0.828637]
VEGETATION_PERCENTAGE    : [3.749158, 15.457872, 9.416382, 27.746645, 6.508834, 26.548102, 20.768043, 1.376445, 0, 1.186114, 10.833184, 3.706451, 24.770588, 3.42564, 20.830637, 15.453044, 24.79527, 19.48252, 25.548872, 17.675065, 0.335142, 17.204528, 28.025195, 16.859227, 27.192914